# 演習5_クラスタリング(k-means)：自動車メーカーの評価

In [ ]:
### 必要なパッケージ（ライブラリ）の読み込み ###
import pandas as pd             # データフレーム型変数を取り扱えるパッケージ"pandas"を読込み（以降"pd"と略記）
import matplotlib.pyplot as plt # グラフ描画のパッケージ"matplotlib"を読込み（以降"plt"と略記）
import seaborn as sns           # 上記matplotlibをベースにした高機能可視化モジュール"saeborn"を読込み（以降"sns"と略記）
import numpy as np              # 数値演算のためのパッケージ"numpy"を読込み（以降"np"と略記）

## データの理解（観察）

### データの読み込み

In [ ]:
# データの読み込み
df = pd.read_excel('ファイル名入力（Tabキーによる文字補完を活用）')

# ※ "演習05_" まで入力し、TABキーで補完するのが効率的

In [ ]:
# データが正しく読み込まれた確認
df

### 参考：描画設定
※重なりなどの描画崩れや、文字化けなど、描画がうまくいかない場合

In [ ]:
### 以下、デザイン設定 ###
# plt.rcParams['figure.figsize'] = 10, 5             # グラフのサイズ指定
plt.rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao',
                                   'IPAexGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP']
                                                   # 文字化け対策にフォント指定
plt.rcParams['font.size'] = 16                     # フォントサイズ一括指定
plt.tight_layout()                                 # グラフ同士が重ならないようにする
# plt.grid(True)                                   # グリッド表示ON
plt.show()                                         # 各種設定の反映

### 散布図行列
※実行に時間がかかるため要注意

In [ ]:
# 散布図行列による傾向確認
sns.pairplot(df.drop(columns=['自動車メーカー'])) #識別番号は可視化から除外

### 文字化け対策にフォント指定 ### ※後段の「描画設定」セル実行でもOK
# sns.set(font='Yu Mincho') # for Win
# sns.set(font='Hiragino Maru Gothic Pro') # for Mac
##############################

plt.show()

### 相関行列の確認

In [ ]:
### 相関行列による変数間の関係性確認 ###
df.corr(numeric_only=True)  # 相関行列の算出

# pandasのv2.0.0以降、数値以外のデータが入っているとエラーになるため、
# 数値データのみに絞り込むオプション numeric_only=True を指定する

In [ ]:
### 相関行列のお化粧（ヒートマップ） ###

colormap = plt.cm.RdBu_r                                   # カラーマップの設定 (RdBu:赤〜青 ⇔ RdBu_r:青〜赤)

plt.rcParams['figure.figsize'] = 10, 5                    # Figureサイズの指定

# # 相関行列の右上半分を隠したい場合は、次の2行のコメントを解除する
# mask = np.zeros_like(df.corr(), dtype=np.bool)
# mask[np.triu_indices_from(mask)] = True

sns.heatmap(df.corr(numeric_only=True),linewidths=0.1, linecolor='white',   # 相関行列df.corr()を引数としヒートマップ作成	
            vmax=1.0, vmin=-1.0, cmap=colormap, annot=True )

            # linewidths/linecolor: 格子線の太さ/色
            # cmap:                 カラーマップの指定
            # vmax/vmin:            最大値/最小値
            # annot:                各要素への数値表示
            # mask:                 可視化から除外する対象（例：mask=mask）
            

## データの準備（加工）

### 欠損値の補完

In [ ]:
### 欠損値の確認 ###
df.isnull().sum()

### ダミー変数化
※今回はカテゴリー値がないので不要

## モデルの構築

### k-meansクラスタリングモデルの構築

In [ ]:
from sklearn.cluster import KMeans # 機械学習パッケージに含まれたKMeansモジュール"KMeans"を読込

# 説明変数のセット
X = df.drop(columns=['自動車メーカー'])  #「自動車メーカー」のカラムを除外した全変数を説明変数にセット

# もしくは、下記のように明示的に説明変数を指定しても可
# X = df[ ['センスがよい', '格調高い', '若々しい', '信頼感のある', '親しみやすい', '先進的な',
#        '技術が優れている', '独創的な', 'スポーティーな', '安心感のある'] ]
#
# X = df.loc[:,'センスがよい':'安心感のある']


# クラスタ数の設定
n = 3

# クラスタリング
model = KMeans( n_clusters=n, random_state=0 )  #クラスタリングの実行準備
                                                # オプションの説明：
                                                # n_clustersでクラスタ数指定
                                                # random_state=数値固定 でモデルの再現性を保持（乱数のシードを固定）
                                                # init='k-means++' or 'random' で初期クラスタ中心の配置方法を指定。defaultは'k-means++'
model.fit(X) #Xを説明変数としてクラスタリング実行

### 結果の出力

In [ ]:
### 所属クラスタ、重心座標の出力 ###
print(model.labels_) # 各データ点が所属するクラスタ
print(model.cluster_centers_) # 各クラスタの重心座標

### 元データへのクラスタ番号紐付け

In [ ]:
### 元のdataframeにクラスタ番号を追加 ###
df['cluster'] = model.labels_

In [ ]:
df #紐付け結果確認

In [ ]:
### 各クラスタに所属するデータ件数確認 ###
df['cluster'].value_counts()

## モデルの評価：クラスタリング結果の可視化

### クラスタごとの分布確認（箱ひげ図）

In [ ]:
# 複数を並べて表示するために、subplotを定義
fig, axes = plt.subplots(3, 4, figsize=(20,10))

# 数値データ（アンケートデータ）のみ抽出（自動車メーカー名とクラスタ番号を除外）
df_num = df.drop(columns=['自動車メーカー','cluster'])

# カラムごとにループを回して、クラスタごとの箱ひげ図を描画
for i, col in enumerate( df_num.columns ):
    j, k = divmod(i, 4) #描くグラフ位置を指定（横方向のグラフ個数=4で割った商と余り）
    sns.boxplot(x='cluster', y=col, data=df, ax=axes[j,k]) #該当するカラムの箱ひげ図を指定の位置に描画
    
#　グラフが重ならないように調整
plt.tight_layout()

### 特定の軸での散布図確認

In [ ]:
### 散布図行列でクラスタごとに色分けして可視化 ###
sns.pairplot( df.drop(columns=['自動車メーカー']), hue='cluster')
# ※実行に時間がかかるため、注意

In [ ]:
plt.rcParams['figure.figsize'] = 10, 7           # グラフのサイズを大きくする
plt.show()

In [ ]:
### 特定の2変数で、クラスタごとに色分けして可視化＋クラスタ重心の可視化 ###

# 特定の2変数を選択
xcol = '親しみやすい'
ycol = '格調高い'

# 特定の2変数でクラスタごとに色分け可視化
sns.scatterplot( data=df, x=xcol, y=ycol, hue='cluster', palette=sns.color_palette(n_colors=3)  )
    #hue: 色分け対象
    #palette: 色パターン
    #クラスタごとにマーカー形状も変えたい場合は、style=df['cluster'] をオプションに加える

# クラスタ重心 (mode.cluster_centers_) の可視化
# ※model.cluster_centers_[:,0]で0番目の説明変数の重心座標
sns.scatterplot( x=model.cluster_centers_[:,1], y=model.cluster_centers_[:,2], marker='*', s=300, color='purple' ) 

# 散布図上に、自動車メーカーの名前をプロット
for i, company in enumerate(df['自動車メーカー']):
    plt.annotate(company, (df[xcol].values[i], df[ycol].values[i]) )

In [ ]:
### 特定の3変数で、クラスタごとに色分けして可視化（3次元プロット）」 ###
### ※環境によっては、うまく表示されない可能性あり ###

from mpl_toolkits.mplot3d.axes3d import Axes3D  #3次元プロットするためのモジュール

#  特定の3変数を選択
xcol = '親しみやすい'
ycol = '格調高い'
zcol = '先進的な'

# グラフを手動で回転させるための設定
%matplotlib notebook

# 3次元グラフの枠を作る
fig = plt.figure()    #枠作成
ax = Axes3D(fig)      #3次元の軸を用意

# データをプロット
colors = {0:'blue', 1:'Orange', 2:'green'}
ax.scatter( data=df, xs=xcol, ys=ycol, zs=zcol, s=50, c=df['cluster'].apply(lambda x: colors[x]) )
  # c=df['cluster']でクラスターごとに色を変更
  # s=50でプロットサイズ指定
  # alpha=1 をオプションに入れると透過性無し
    
# 軸ラベルの表示
ax.set_xlabel( xcol )
ax.set_ylabel( ycol )
ax.set_zlabel( zcol )

#fig.colorbar(p) # カラーバーを表示

plt.show()

In [ ]:
# グラフをインライン表示に戻す（3次元プロット後は、必ず実行しておくこと！）
# ※"%matplotlib notebook"の表示設定のままだと、3Dグラフ以外はうまく表示されないので元に戻す

%matplotlib inline


### 最適なクラスタ数の確認（Elbow法）

In [ ]:
SSE = []                      #クラスタ評価関数（クラスタ内誤差平方和: SSE）を格納するリストを準備

for i in range(1,7):          #クラスタ数を1から6まで振って順次k-meansを実行：range(start, stop)で start ≦ i < stop の連番を生成
    model = KMeans(n_clusters=i, random_state=0) #クラスタリングの実行準備
    model.fit(X)                                 #Xを説明変数としてk-meansクラスタリングの計算を実行
    SSE.append(model.inertia_)                   #各クラスタ内のクラスタ内誤差平方和を取得し、リストに格納 ※inertiaは慣性という意味
                                                 #（各データが所属クラスタの中心からどれだけ離れているか）
plt.plot(range(1,7), SSE, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE (Sum of Squared Errors)')
plt.show()